# Automated ML with Regression model
## On case of Taxi Data

## Flow

In [ ]:
from azureml.opendatasets import NycTlcGreen
import pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta

## Getting Taxi Data

In [2]:
green_taxi_df = pd.DataFrame([])
start = datetime.strptime("1/1/2015","%m/%d/%Y")
end = datetime.strptime("1/31/2015","%m/%d/%Y")

for sample_month in range(12):
    temp_df_green = NycTlcGreen(start + relativedelta(months=sample_month), end + relativedelta(months=sample_month)) \
        .to_pandas_dataframe()
    green_taxi_df = green_taxi_df.append(temp_df_green.sample(2000))

green_taxi_df.head(10)

ActivityStarted, to_pandas_dataframe
ActivityStarted, to_pandas_dataframe_in_worker
Target paths: ['/puYear=2015/puMonth=1/']
Looking for parquet files...
Reading them into Pandas dataframe...
Reading green/puYear=2015/puMonth=1/part-00175-tid-3144095708418575143-7b1fb9db-f589-44c3-b42e-832eaf9f427c-21996.c000.snappy.parquet under container nyctlc
Done.
ActivityCompleted: Activity=to_pandas_dataframe_in_worker, HowEnded=Success, Duration=7914.28 [ms]
ActivityCompleted: Activity=to_pandas_dataframe, HowEnded=Success, Duration=7925.96 [ms]
ActivityStarted, to_pandas_dataframe
ActivityStarted, to_pandas_dataframe_in_worker
Target paths: ['/puYear=2015/puMonth=2/']
Looking for parquet files...
Reading them into Pandas dataframe...
Reading green/puYear=2015/puMonth=2/part-00007-tid-3144095708418575143-7b1fb9db-f589-44c3-b42e-832eaf9f427c-21828.c000.snappy.parquet under container nyctlc
Done.
ActivityCompleted: Activity=to_pandas_dataframe_in_worker, HowEnded=Success, Duration=9138.71 [ms]
A

,vendorID,lpepPickupDatetime,lpepDropoffDatetime,passengerCount,tripDistance,puLocationId,doLocationId,pickupLongitude,pickupLatitude,dropoffLongitude,...,paymentType,fareAmount,extra,mtaTax,improvementSurcharge,tipAmount,tollsAmount,ehailFee,totalAmount,tripType
987815,1,2015-01-11 00:24:42,2015-01-11 00:31:32,2,0.80,None,None,-73.957893,40.717674,-73.966057,...,1,6.0,0.5,0.5,0.3,1.45,0.00,NaN,8.75,1.0
1289743,2,2015-01-01 07:21:23,2015-01-01 07:44:43,1,18.86,None,None,-73.918915,40.765869,-73.643799,...,2,50.0,0.0,0.5,0.3,0.00,0.00,NaN,50.80,1.0
472246,1,2015-01-23 20:21:06,2015-01-23 20:26:06,1,0.90,None,None,-73.966438,40.804077,-73.954735,...,2,6.0,0.5,0.5,0.3,0.00,0.00,NaN,7.30,1.0
1320313,2,2015-01-15 17:13:29,2015-01-15 17:17:48,1,0.45,None,None,-73.916916,40.761421,-73.910362,...,2,4.5,1.0,0.5,0.3,0.00,0.00,NaN,6.30,1.0
1377752,2,2015-01-16 19:53:44,2015-01-16 20:02:21,1,1.37,None,None,-73.977859,40.684811,-73.995529,...,2,8.0,1.0,0.5,0.3,0.00,0.00,NaN,9.80,1.0
895263,2,2015-01-06 22:35:33,2015-01-06 22:49:30,2,6.60,None,None,-73.958611,40.815624,-73.914711,...,2,20.5,0.5,0.5,0.3,0.00,2.44,NaN,24.24,1.0
1138116,2,2015-01-20 19:21:12,2015-01-20 19:38:00,1,5.27,None,None,-73.985718,40.689564,-73.959816,...,1,18.0,1.0,0.5,0.3,2.00,0.00,NaN,21.80,1.0
898900,1,2015-01-07 00:15:27,2015-01-07 00:31:40,1,3.90,None,None,-73.900787,40.862614,-73.905113,...,1,14.5,0.5,0.5,0,0.00,0.00,NaN,15.50,1.0
36138,2,2015-01-08 15:57:45,2015-01-08 16:05:17,1,0.75,None,None,-73.998367,40.681953,-73.990456,...,2,6.5,1.0,0.5,0.3,0.00,0.00,NaN,8.30,1.0
266807,1,2015-01-14 12:53:46,2015-01-14 13:02:48,1,1.70,None,None,-73.947319,40.674374,-73.956650,...,1,8.5,0.0,0.5,0.3,1.00,0.00,NaN,10.30,1.0


## Creating different Time Various frames

In [3]:
def build_time_features(vector):
    pickup_datetime = vector[0]
    month_num = pickup_datetime.month
    day_of_month = pickup_datetime.day
    day_of_week = pickup_datetime.weekday()
    hour_of_day = pickup_datetime.hour

    return pd.Series((month_num, day_of_month, day_of_week, hour_of_day))

green_taxi_df[["month_num", "day_of_month","day_of_week", "hour_of_day"]] = green_taxi_df[["lpepPickupDatetime"]].apply(build_time_features, axis=1)
green_taxi_df.head(10)

,vendorID,lpepPickupDatetime,lpepDropoffDatetime,passengerCount,tripDistance,puLocationId,doLocationId,pickupLongitude,pickupLatitude,dropoffLongitude,...,improvementSurcharge,tipAmount,tollsAmount,ehailFee,totalAmount,tripType,month_num,day_of_month,day_of_week,hour_of_day
987815,1,2015-01-11 00:24:42,2015-01-11 00:31:32,2,0.80,None,None,-73.957893,40.717674,-73.966057,...,0.3,1.45,0.00,NaN,8.75,1.0,1,11,6,0
1289743,2,2015-01-01 07:21:23,2015-01-01 07:44:43,1,18.86,None,None,-73.918915,40.765869,-73.643799,...,0.3,0.00,0.00,NaN,50.80,1.0,1,1,3,7
472246,1,2015-01-23 20:21:06,2015-01-23 20:26:06,1,0.90,None,None,-73.966438,40.804077,-73.954735,...,0.3,0.00,0.00,NaN,7.30,1.0,1,23,4,20
1320313,2,2015-01-15 17:13:29,2015-01-15 17:17:48,1,0.45,None,None,-73.916916,40.761421,-73.910362,...,0.3,0.00,0.00,NaN,6.30,1.0,1,15,3,17
1377752,2,2015-01-16 19:53:44,2015-01-16 20:02:21,1,1.37,None,None,-73.977859,40.684811,-73.995529,...,0.3,0.00,0.00,NaN,9.80,1.0,1,16,4,19
895263,2,2015-01-06 22:35:33,2015-01-06 22:49:30,2,6.60,None,None,-73.958611,40.815624,-73.914711,...,0.3,0.00,2.44,NaN,24.24,1.0,1,6,1,22
1138116,2,2015-01-20 19:21:12,2015-01-20 19:38:00,1,5.27,None,None,-73.985718,40.689564,-73.959816,...,0.3,2.00,0.00,NaN,21.80,1.0,1,20,1,19
898900,1,2015-01-07 00:15:27,2015-01-07 00:31:40,1,3.90,None,None,-73.900787,40.862614,-73.905113,...,0,0.00,0.00,NaN,15.50,1.0,1,7,2,0
36138,2,2015-01-08 15:57:45,2015-01-08 16:05:17,1,0.75,None,None,-73.998367,40.681953,-73.990456,...,0.3,0.00,0.00,NaN,8.30,1.0,1,8,3,15
266807,1,2015-01-14 12:53:46,2015-01-14 13:02:48,1,1.70,None,None,-73.947319,40.674374,-73.956650,...,0.3,1.00,0.00,NaN,10.30,1.0,1,14,2,12


## Remove unneeded columns and clean data

In [4]:
columns_to_remove = ["lpepPickupDatetime", "lpepDropoffDatetime", "puLocationId", "doLocationId", "extra", "mtaTax",
                     "improvementSurcharge", "tollsAmount", "ehailFee", "tripType", "rateCodeID",
                     "storeAndFwdFlag", "paymentType", "fareAmount", "tipAmount"
                    ]
for col in columns_to_remove:
    green_taxi_df.pop(col)

green_taxi_df.head(5)

,vendorID,passengerCount,tripDistance,pickupLongitude,pickupLatitude,dropoffLongitude,dropoffLatitude,totalAmount,month_num,day_of_month,day_of_week,hour_of_day
987815,1,2,0.80,-73.957893,40.717674,-73.966057,40.714550,8.75,1,11,6,0
1289743,2,1,18.86,-73.918915,40.765869,-73.643799,40.793125,50.80,1,1,3,7
472246,1,1,0.90,-73.966438,40.804077,-73.954735,40.808006,7.30,1,23,4,20
1320313,2,1,0.45,-73.916916,40.761421,-73.910362,40.761711,6.30,1,15,3,17
1377752,2,1,1.37,-73.977859,40.684811,-73.995529,40.683136,9.80,1,16,4,19


### Finalize the data

In [8]:
final_df = green_taxi_df.query("pickupLatitude>=40.53 and pickupLatitude<=40.88")
final_df = final_df.query("pickupLongitude>=-74.09 and pickupLongitude<=-73.72")
final_df = final_df.query("tripDistance>=0.25 and tripDistance<31")
final_df = final_df.query("passengerCount>0 and totalAmount>0")

columns_to_remove_for_training = ["pickupLongitude", "pickupLatitude", "dropoffLongitude", "dropoffLatitude"]
for col in columns_to_remove_for_training:
    final_df.pop(col)

## Check some descriptive statistics

In [9]:
green_taxi_df.describe()

,vendorID,passengerCount,tripDistance,pickupLongitude,pickupLatitude,dropoffLongitude,dropoffLatitude,totalAmount,month_num,day_of_month,day_of_week,hour_of_day
count,24000.000000,24000.000000,24000.000000,24000.000000,24000.000000,24000.000000,24000.000000,24000.000000,24000.000000,24000.000000,24000.000000,24000.000000
mean,1.785542,1.383125,2.872745,-73.827204,40.689827,-73.820145,40.684644,14.744525,6.500000,15.108750,3.270667,13.521292
std,0.410455,1.064137,2.925808,2.821817,1.556134,2.901286,1.599803,11.377393,3.452124,8.492277,1.966931,6.837945
min,1.000000,0.000000,0.000000,-74.177345,0.000000,-74.214874,0.000000,-150.300000,1.000000,1.000000,0.000000,0.000000
25%,2.000000,1.000000,1.050000,-73.959490,40.699130,-73.967392,40.699615,7.800000,3.750000,8.000000,2.000000,9.000000
50%,2.000000,1.000000,1.900000,-73.945351,40.746767,-73.944927,40.747662,11.300000,6.500000,15.000000,3.000000,15.000000
75%,2.000000,1.000000,3.600000,-73.917377,40.802999,-73.909475,40.792078,17.800000,9.250000,22.000000,5.000000,19.000000
max,2.000000,8.000000,62.210000,0.000000,42.324314,0.000000,42.324368,300.000000,12.000000,30.000000,6.000000,23.000000


## Configure WorkSpace

In [6]:
from azureml.core.workspace import Workspace
ws = Workspace.from_config()

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code CP9WU585S to authenticate.
Interactive authentication successfully completed.


## Split the data into train and test sets

In [10]:
from sklearn.model_selection import train_test_split

y_df = final_df.pop("totalAmount")
x_df = final_df

x_train, x_test, y_train, y_test = train_test_split(x_df, y_df, test_size=0.2, random_state=223)

## Automatically train a model
### Setup the logging and specify:

Property Value in this tutorial Description iteration_timeout_minutes 2 Time limit in minutes for each iteration. Reduce this value to decrease total runtime. iterations 20 Number of iterations. In each iteration, a new machine learning model is trained with your data. This is the primary value that affects total run time. primary_metric spearman_correlation Metric that you want to optimize. The best-fit model will be chosen based on this metric. preprocess True By using True, the experiment can preprocess the input data (handling missing data, converting text to numeric, etc.) verbosity logging.INFO Controls the level of logging. n_cross_validations 5 Number of cross-validation splits to perform when validation data is not specified.


In [11]:
import logging

automl_settings = {
    "iteration_timeout_minutes": 2,
    "iterations": 20,
    "primary_metric": 'spearman_correlation',
    "preprocess": True,
    "verbosity": logging.INFO,
    "n_cross_validations": 5
}

In [12]:
from azureml.train.automl import AutoMLConfig

automl_config = AutoMLConfig(task='regression',
                             debug_log='automated_ml_errors.log',
                             X=x_train.values,
                             y=y_train.values.flatten(),
                             **automl_settings)

WARNING - From /anaconda/envs/azureml_py36/lib/python3.6/site-packages/azureml/automl/core/_vendor/automl/client/core/common/tf_wrappers.py:36: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.

WARNING - From /anaconda/envs/azureml_py36/lib/python3.6/site-packages/azureml/automl/core/_vendor/automl/client/core/common/tf_wrappers.py:36: The name tf.logging.ERROR is deprecated. Please use tf.compat.v1.logging.ERROR instead.



### Train the automatic regression model

In [ ]:
from azureml.core.experiment import Experiment
experiment = Experiment(ws, "taxi-experiment")
local_run = experiment.submit(automl_config, show_output=True)

Running on local machine
Parent Run ID: AutoML_fe26a99f-64e0-4620-824f-f5c9076a73c2
Current status: DatasetFeaturization. Beginning to featurize the dataset.
Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturizationCompleted. Completed featurizing the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.

****************************************************************************************************
DATA GUARDRAILS SUMMARY:
For more details, use API: run.get_guardrails()

TYPE:         Missing Values Imputation
STATUS:       PASSED
DESCRIPTION:  There were no missing values found in the training data.

TYPE:         High Cardinality Feature Detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and no high cardinality features were detected.

**************************************************************

### Explore the results

Explore the results of automatic training with a Jupyter widget. The widget allows you to see a graph and table of all individual run iterations, along with training accuracy metrics and metadata. Additionally, you can filter on different accuracy metrics than your primary metric with the dropdown selector.



In [ ]:
from azureml.widgets import RunDetails
RunDetails(local_run).show()

### Retrieve the best model

In [ ]:
best_run, fitted_model = local_run.get_output()
print(best_run)
print(fitted_model)

### Test the best model accuracy

Use the best model to run predictions on the test data set to predict taxi fares. The function predict uses the best model and predicts the values of y, trip cost, from the x_test data set. Print the first 10 predicted cost values from y_predict.

In [ ]:
y_predict = fitted_model.predict(x_test.values)
print(y_predict[:10])

In [ ]:
from sklearn.metrics import mean_squared_error
from math import sqrt

y_actual = y_test.values.flatten().tolist()
rmse = sqrt(mean_squared_error(y_actual, y_predict))
rmse

## Calculating MAPE

In [ ]:
sum_actuals = sum_errors = 0

for actual_val, predict_val in zip(y_actual, y_predict):
    abs_error = actual_val - predict_val
    if abs_error < 0:
        abs_error = abs_error * -1

    sum_errors = sum_errors + abs_error
    sum_actuals = sum_actuals + actual_val

mean_abs_percent_error = sum_errors / sum_actuals
print("Model MAPE:")
print(mean_abs_percent_error)
print()
print("Model Accuracy:")
print(1 - mean_abs_percent_error)